# Imports

In [2]:
from loader import read_obsmat, read_groups

# Load data

In [3]:
dataset_path = './UCY/students03'
df = read_obsmat(dataset_path)
groups = read_groups(dataset_path)

In [4]:
df.head()

,frame_id,agent_id,pos_x,pos_y,v_x,v_y,timestamp
0,1,1,1.641300,2.188291,-0.161799,0.702095,0.04
1,1,2,0.222933,4.477681,0.005909,0.711261,0.04
2,1,3,3.873701,-0.476888,-0.703324,0.454779,0.04
3,1,4,1.665726,-4.184461,-0.315806,0.378670,0.04
4,1,5,1.030742,6.572421,-0.142576,-1.387602,0.04


In [5]:
print('Shape of dataframe: {}'.format(df.shape))
print('Number of unique agents: {}'.format(df.agent_id.unique().size))
print('Number of unique frames: {}'.format(df.frame_id.unique().size))

Shape of dataframe: (21846, 7)
Number of unique agents: 428
Number of unique frames: 540


In [7]:
print('Number of groups: {}'.format(len(groups)))

from collections import Counter
count_dict = Counter([agent for group in groups for agent in group])
multigroup_agents = [key for key, value in count_dict.items() if value > 1]
print('Agents {} are found in more than one group'.format(multigroup_agents))

group_agents = [agent for agent in count_dict.elements()]
print('Number of agents in groups: {}'.format(len(group_agents)))

Number of groups: 103
Agents ['', '421', '422', '121'] are found in more than one group
Number of agents in groups: 542


number of frames for each agent

In [54]:
df['agent_id'].value_counts()

468    539
473    539
233    288
232    287
228    263
      ... 
407      6
369      6
299      5
298      5
408      2
Name: agent_id, Length: 428, dtype: int64

number of people for each frame

In [55]:
df['frame_id'].value_counts()
' 6    18   421   422   423   425 '

1001    62
981     62
991     62
1011    60
1021    59
        ..
4951    22
4941    22
4961    22
4931    21
5391    16
Name: frame_id, Length: 540, dtype: int64

In [22]:
from loader import load_eth

In [23]:
eth_traj_dataset = load_eth('./ETH/seq_eth/obsmat.txt')
eth_trajs = eth_traj_dataset.get_trajectories()

In [25]:
eth_traj_dataset.data.iloc[eth_trajs.groups[0, 1]] # get trajectory of scene_id 0 and pedestrian_id 1

,frame_id,agent_id,pos_x,pos_y,vel_x,vel_y,scene_id,label,timestamp
0,780,1,8.456844,3.588066,1.671714,0.176292,0,pedestrian,52.0
1,786,1,9.125530,3.658583,1.662877,0.326723,0,pedestrian,52.4
2,792,1,9.787146,3.849445,1.683334,0.371084,0,pedestrian,52.8
3,798,1,10.472197,3.955450,1.598568,0.264795,0,pedestrian,53.2
4,804,1,11.066000,4.061280,1.574526,0.456390,0,pedestrian,53.6
6,810,1,11.731818,4.320563,1.644127,0.544391,0,pedestrian,54.0
8,816,1,12.381302,4.496793,1.623709,0.440576,0,pedestrian,54.4


In [26]:
eth_traj_dataset.data[eth_traj_dataset.data['agent_id'] == 1]

,frame_id,agent_id,pos_x,pos_y,vel_x,vel_y,scene_id,label,timestamp
0,780,1,8.456844,3.588066,1.671714,0.176292,0,pedestrian,52.0
1,786,1,9.125530,3.658583,1.662877,0.326723,0,pedestrian,52.4
2,792,1,9.787146,3.849445,1.683334,0.371084,0,pedestrian,52.8
3,798,1,10.472197,3.955450,1.598568,0.264795,0,pedestrian,53.2
4,804,1,11.066000,4.061280,1.574526,0.456390,0,pedestrian,53.6
6,810,1,11.731818,4.320563,1.644127,0.544391,0,pedestrian,54.0
8,816,1,12.381302,4.496793,1.623709,0.440576,0,pedestrian,54.4


In [27]:
eth_traj_dataset.data.shape

(8908, 9)

# Thoughts and TODOs

goal: data for all agents, both in or not in groups
problems: agents in low number of frames

number of same agents in X consecutive frames

In [13]:
# TODO calculate number of same agents in all possible X consecutive frames, find set of agents in each frame, check intersection between the sets

agents that exist in low number of frames

In [20]:
# TODO for agents with lower than X frames, generate data to reach X frames

time window 1 frame

In [ ]:
# TODO check number of data when time window is 1 frame

agents in multiple groups

In [ ]:
# TODO check number of agents in multiple groups